In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/machihe learning/Projects/Recomendation system/products.csv')

In [3]:
df.head(3)

,product_id,product_name,product_category,product_description,price
0,1,Politics Forget,Toys & Games,Difficult trip live own skin that. At use prod...,81.98
1,2,These Care,Toys & Games,Meeting knowledge center decade notice. Federa...,35.39
2,3,Career Decide,Beauty,Use week American address artist explain face....,445.48


In [4]:
# Load the datasets
try:
    interactions_df = pd.read_csv('C:/machihe learning/Projects/Recomendation system/interactions.csv')
    products_df = pd.read_csv('C:/machihe learning/Projects/Recomendation system/products.csv')
    users_df = pd.read_csv('C:/machihe learning/Projects/Recomendation system/users.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: One or more CSV files not found. Make sure they are in the correct directory.")

Data loaded successfully.


In [5]:
# --- Perform initial inspection ---
print("\nInteractions DataFrame Info:")
interactions_df.info()


Interactions DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   product_id  100000 non-null  int64 
 2   event_type  100000 non-null  object
 3   timestamp   100000 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.1+ MB


In [6]:
# --- Merge the DataFrames ---
# We'll merge interactions with products to get product details
merged_df = pd.merge(interactions_df, products_df, on='product_id')

In [7]:
# Now, we'll merge with users to get user details
final_df = pd.merge(merged_df, users_df, on='user_id')

In [8]:
print("\nFinal Merged DataFrame Info:")
final_df.info()


Final Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              100000 non-null  int64  
 1   product_id           100000 non-null  int64  
 2   event_type           100000 non-null  object 
 3   timestamp            100000 non-null  object 
 4   product_name         100000 non-null  object 
 5   product_category     100000 non-null  object 
 6   product_description  100000 non-null  object 
 7   price                100000 non-null  float64
 8   signup_date          100000 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 6.9+ MB


In [9]:
# --- Filter for 'purchase' events as our main signal ---
purchases_df = final_df[final_df['event_type'] == 'purchase'].copy()

In [10]:
print("\nPurchases DataFrame Info:")
purchases_df.info()
print(purchases_df.head())


Purchases DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 9863 entries, 20 to 99983
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              9863 non-null   int64  
 1   product_id           9863 non-null   int64  
 2   event_type           9863 non-null   object 
 3   timestamp            9863 non-null   object 
 4   product_name         9863 non-null   object 
 5   product_category     9863 non-null   object 
 6   product_description  9863 non-null   object 
 7   price                9863 non-null   float64
 8   signup_date          9863 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 770.5+ KB
    user_id  product_id event_type            timestamp  \
20      952        3899   purchase  2024-05-04 12:42:07   
36       75         158   purchase  2024-02-14 11:40:39   
41      929        3799   purchase  2024-02-05 06:54:03   
47       48        2

In [11]:
import os

os.makedirs('data/processed', exist_ok=True)
purchases_df.to_csv('data/processed/purchases_data.csv', index=False)

In [12]:
purchases_df.head(3)

,user_id,product_id,event_type,timestamp,product_name,product_category,product_description,price,signup_date
20,952,3899,purchase,2024-05-04 12:42:07,Reason Much,Sports,History instead seek bad including. Material i...,438.70,2023-04-21
36,75,158,purchase,2024-02-14 11:40:39,Campaign The,Home & Kitchen,Different material political value crime bette...,271.53,2020-09-24
41,929,3799,purchase,2024-02-05 06:54:03,Identify Field,Beauty,Ok growth mention ready bad. Pay necessary cul...,411.61,2023-06-27


# Build the model

In [13]:
pip install scikit-surprise

You should consider upgrading via the 'c:\machihe learning\Projects\Recomendation system\recom39\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [15]:
# Load the processed data
purchases_df = pd.read_csv('data/processed/purchases_data.csv')

In [16]:
# We can just assign a value of 1 since a purchase is a positive signal.
purchases_df['rating'] = 1

reader = Reader(rating_scale=(1, 1)) # We use (1,1) since all our ratings are 1

In [17]:
# Load the DataFrame into the Surprise Dataset format
data = Dataset.load_from_df(purchases_df[['user_id', 'product_id', 'rating']], reader)

# Train the model

In [19]:
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

print("Model training complete.")

Model training complete.


In [20]:
# --- 3. Make a prediction for a specific user ---

In [21]:
# Let's pick a random user to make a prediction for.
user_id_to_predict = purchases_df['user_id'].iloc[0] # Get the first user from the DataFrame
product_id_to_predict = products_df[~products_df['product_id'].isin(
    purchases_df[purchases_df['user_id'] == user_id_to_predict]['product_id']
)]['product_id'].iloc[0]

# Predict the rating the user would give to the product
prediction = model.predict(user_id_to_predict, product_id_to_predict)

print(f"\nPredicted rating for user {user_id_to_predict} on product {product_id_to_predict}: {prediction.est}")


Predicted rating for user 952 on product 1: 1


In [22]:
# --- 4. Get Top N recommendations for a user ---
# This is a more practical application.
def get_top_n_recommendations(model, user_id, all_products, k=10):
    # Get products the user has NOT purchased
    user_purchased_items = purchases_df[purchases_df['user_id'] == user_id]['product_id']
    unseen_products = [product for product in all_products if product not in user_purchased_items.values]

    # Predict ratings for all unseen products
    predictions = [model.predict(user_id, product_id) for product_id in unseen_products]

    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top k products
    top_k_products = predictions[:k]
    
    return [pred.iid for pred in top_k_products]

# Get all unique product IDs
all_products = products_df['product_id'].unique()

top_recommendations = get_top_n_recommendations(model, user_id_to_predict, all_products)
print(f"\nTop 10 recommended product IDs for user {user_id_to_predict}:")
print(top_recommendations)


Top 10 recommended product IDs for user 952:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# Evaluating

In [23]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [24]:
# --- Train the model on the training set ---
model = SVD()
model.fit(trainset)

In [25]:
# --- Make predictions on the testing set ---
predictions = model.test(testset)

In [26]:
rmse = accuracy.rmse(predictions)
print(f"The model's RMSE on the test set is: {rmse}")

RMSE: 0.0000
The model's RMSE on the test set is: 0.0


In [27]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=10)